In [127]:
import os
import time
from stat import S_ISREG, ST_CTIME, ST_MODE
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd


ModuleNotFoundError: No module named 'print_options'

In [42]:
# now read in a csv
data_file_name='csv/MGA6_9.csv'
df=pd.read_csv(data_file_name)
#dfiltindex=df.loc[df['parent']!=df['parent'].shift(-1)].index
dfb=df.loc[df['wasFitter']]  # filter the ones that are fitter
dfw=df.loc[~df['wasFitter']] # filter the ones that aren't fitter
arch='play.M'
ind_succ=dfb.loc[~(dfb['parent'].str.contains(arch))].index
ind_unsucc=dfw.loc[~(dfw['parent'].str.contains(arch))].index
#df[pd.concat([df.category.shift(-i)=='A' for i in range(n)], axis=1).any(axis=1)]
#idx = df.index.get_indexer_for(df[df.category=='parent'].index)
print(ind_succ)
print(ind_unsucc)

Int64Index([351, 1517, 1518, 1520, 1522, 1524, 1526, 1528], dtype='int64')
Int64Index([0, 115, 229, 352, 521, 785, 1163, 1530, 1971], dtype='int64')


In [43]:
# extract the indices around success
n=20
succ_after=df.iloc[np.unique(np.concatenate([np.arange(max(i,0), min(i+n+1, len(df))) for i in ind_succ]))]
succ_before=df.iloc[np.unique(np.concatenate([np.arange(max(i-n,0), min(i, len(df))) for i in ind_succ]))]
energy_after_succ=succ_after[['joules']]
energy_before_succ=succ_before[['joules']]
print(energy_after_succ)
print('before successful immigrant')
print(energy_before_succ['joules'].mean())
print('after successful immigrant')
print(energy_after_succ['joules'].mean())

        joules
351   15.53112
352   14.87844
353   14.79852
354   14.94504
355   18.10188
356   14.87844
357   16.79652
358   14.63868
359   14.91840
360   14.27904
361   18.10188
362   14.62536
363   18.02196
364   14.78520
365   14.71860
366   14.82516
367   16.72992
368   14.81184
369   20.15316
370   14.63868
371   14.53212
1517   3.83616
1518   3.63636
1519   4.40892
1520   3.72960
1521   4.46220
1522   3.51648
1523   4.98168
1524   3.43656
1525   3.90276
1526   3.31668
1527   4.18248
1528   3.38328
1529   3.72960
1530   3.60972
1531   3.54312
1532   3.48984
1533   4.06260
1534   3.40992
1535   3.55644
1536   3.40992
1537   4.00932
1538   3.64968
1539   3.99600
1540   3.51648
1541   3.79620
1542   3.75624
1543   4.44888
1544   3.51648
1545   4.40892
1546   3.79620
1547   3.59640
1548   3.36996
before successful immigrant
8.724338823529411
after successful immigrant
8.512736603773586


In [46]:
n=20
unsucc_after=df.iloc[np.unique(np.concatenate([np.arange(max(i,0), min(i+n+1, len(df))) for i in ind_unsucc]))]
unsucc_after_idxs=np.unique(np.concatenate([np.arange(max(i,0), min(i+n+1, len(df))) for i in ind_unsucc]))
print(unsucc_after_idxs)
unsucc_before=df.iloc[np.unique(np.concatenate([np.arange(max(i-n,0), min(i, len(df))) for i in ind_unsucc]))]
energy_after_unsucc=unsucc_after[['joules']]
energy_before_unsucc=unsucc_before[['joules']]
print(energy_before_unsucc)
print('before successful immigrant')
print(energy_before_unsucc['joules'].mean())
print('after successful immigrant')
print(energy_after_unsucc['joules'].mean())

[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20  115  116  117  118  119  120  121
  122  123  124  125  126  127  128  129  130  131  132  133  134  135
  229  230  231  232  233  234  235  236  237  238  239  240  241  242
  243  244  245  246  247  248  249  352  353  354  355  356  357  358
  359  360  361  362  363  364  365  366  367  368  369  370  371  372
  521  522  523  524  525  526  527  528  529  530  531  532  533  534
  535  536  537  538  539  540  541  785  786  787  788  789  790  791
  792  793  794  795  796  797  798  799  800  801  802  803  804  805
 1163 1164 1165 1166 1167 1168 1169 1170 1171 1172 1173 1174 1175 1176
 1177 1178 1179 1180 1181 1182 1183 1530 1531 1532 1533 1534 1535 1536
 1537 1538 1539 1540 1541 1542 1543 1544 1545 1546 1547 1548 1549 1550
 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984
 1985 1986 1987 1988 1989 1990 1991]
        joules
95    14.83848
96    14.6

In [134]:
def collate_before_after_inds(inds,df):
    befores=[] # list of lists of before energy readings
    before_idxs=np.unique(np.concatenate([np.arange(max(i-n,0), min(i, len(df))) for i in inds]))
    afters=[] # lists of lists of after energy readings
    after_idxs=np.unique(np.concatenate([np.arange(max(i,0), min(i+n+1, len(df))) for i in inds]))
    
    # collate the before readings
    curr_readings=[] # seed the readings
    idx=0 # index of the readings
    while True:
        # if we are just about to run out if indices
        if idx >= (len(before_idxs)-2):
            last_rows=df.iloc[before_idxs[idx]:before_idxs[idx+1]]
            last_vals=list(last_rows['joules'].array)
            curr_readings+=last_vals
            befores.append(curr_readings) # finish off lists
            break
        this_row=df.iloc[before_idxs[idx]]
        curr_reading=this_row['joules']
        #print('curr_reading'+ str(curr_reading))
        curr_readings+=list([curr_reading])
        if (abs(before_idxs[idx+1]-before_idxs[idx])>1):
            #start new list
            #print('starting new list')
            befores.append(curr_readings)
            curr_readings=[] # reinitialise lists
        idx+=1
        
    # collate the after readings
    curr_readings=[] # seed the readings
    idx=0 # index of the readings
    while True:
        # if we are just about to run out if indices
        if idx >= (len(after_idxs)-2):
            last_rows=df.iloc[after_idxs[idx]:after_idxs[idx+1]]
            last_vals=list(last_rows['joules'].array)
            curr_readings+=last_vals
            afters.append(curr_readings) # finish off lists
            break
        this_row=df.iloc[after_idxs[idx]]
        curr_reading=this_row['joules']
        #print('curr_reading'+ str(curr_reading))
        curr_readings+=list([curr_reading])
        if (abs(after_idxs[idx+1]-after_idxs[idx])>1):
            #start new list
            #print('starting new list')
            afters.append(curr_readings)
            curr_readings=[] # reinitialise lists
        idx+=1     
        
        
        
        
    return zip(befores,afters)
            
        

def collect_before_after_for_project(proj_name,arch):
    succs=[] # list of energy readings before/after successful immigrants
    unsuccs=[] # list of energy readings befor/after unsucessful immigrants
    # for each experimental run
    for i in range(1,11):
        data_file_name='csv/'+proj_name+'_'+str(i)+'.csv'
        df=pd.read_csv(data_file_name)
        dfb=df.loc[df['wasFitter']]  # filter the ones that are fitter
        dfw=df.loc[~df['wasFitter']] # filter the ones that aren't fitter
        ind_succ=dfb.loc[~(dfb['parent'].str.contains(arch))].index
        ind_unsucc=dfw.loc[~(dfw['parent'].str.contains(arch))].index
        #print(ind_succ)
        succs+=collate_before_after_inds(ind_succ,df)
        #print(ind_unsucc)
        unsuccs+=collate_before_after_inds(ind_unsucc,df)
    return (succs,unsuccs)

def collate_means_to_points(means_array):
    # takes in a list of pairs of lists and converts to a list of pairs consisting of the mean of each 
    # array
    res=[(sum(xs)/len(xs),sum(ys)/len(ys)) for (xs,ys) in means_array]
    return res

def print_before_after(means_array):
    # pretty print
    print([(["{0:0.2f}".format(i) for i in xs],["{0:0.2f}".format(i) for i in ys])for (xs,ys) in means_array])

In [135]:
(succs,unsuccs)=collect_before_after_for_project('MGA6','play.M')
print_before_after(succs)
means=collate_means_to_points(succs)
xs=[x for (x,y) in means]
ys=[y for (x,y) in means]

[(['28.32', '18.47', '18.78', '18.79', '22.86', '18.74', '18.73', '18.45', '18.63', '18.42', '22.90', '18.47', '18.90', '18.50', '18.71', '18.77', '18.77', '18.67', '18.87', '18.77', '15.22', '14.57'], ['15.22', '14.57', '14.92', '14.80', '15.00', '14.43', '14.40', '14.40', '22.03', '14.69', '15.81', '14.57', '14.47', '14.44', '17.94', '14.43', '14.61', '14.52', '14.68', '14.27', '18.32', '14.56', '17.60']), (['21.84', '22.04', '22.64', '22.52', '22.20', '22.44', '25.61', '22.24', '22.47', '21.84', '22.54', '22.46', '22.55', '22.30', '24.95', '22.27', '22.76', '22.08', '21.99', '22.55', '20.51'], ['33.89', '35.44', '34.97', '34.89', '34.62', '39.16', '34.78', '31.25', '30.21', '30.06', '31.74', '27.01', '26.73', '27.04', '26.88', '26.51', '26.88', '31.71', '26.33', '25.76', '25.73']), (['18.61', '19.07', '18.41', '18.74', '18.28', '18.53', '18.51', '18.71', '18.62', '19.87', '18.67', '18.58', '18.61', '18.17', '23.26', '18.59', '18.99', '18.28', '23.67', '18.62', '3.70', '3.25', '3.26'

NameError: name 'unzip' is not defined